In [ ]:
from utils import mnist_reader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
import random


xtr, ytr = mnist_reader.load_mnist('data/fashion', kind='train')
xte, yte = mnist_reader.load_mnist('data/fashion', kind='t10k')


ytr_noisy = ytr.copy()
for i in range(len(ytr_noisy)):
    if random.random() < 0.2:
        ytr_noisy[i] = random.randint(0, 9)



In [ ]:
counter = {}
for i in ytr:
    counter[i] = counter.get(i, 0) + 1
print(counter)


{9: 6000, 0: 6000, 3: 6000, 2: 6000, 7: 6000, 5: 6000, 1: 6000, 6: 6000, 4: 6000, 8: 6000}


In [ ]:

lsvm = {}
gsvm = {}

for i in range(10):
    lsvm[10**i * 0.0001] = LinearSVC(C=(10**i * 0.0001))
    for j in range(10):
        gsvm[(10**i * 0.0001, 10**j * 0.0001)] =  SVC(C=(10**i * 0.0001), gamma=(10**j * 0.0001))
    print(f"{i + 1}/10 done")

1/10 done
2/10 done
3/10 done
4/10 done
5/10 done
6/10 done
7/10 done
8/10 done
9/10 done
10/10 done


In [ ]:
neural = {}
activations = ['identity', 'logistic', 'tanh', 'relu']

for i in range(1, 8):
    for j in activations:
        neural[(2**i, j)] = MLPClassifier(hidden_layer_sizes=(2**i, ), activation=j)
        for k in range(1, 8):
            neural[(2**i, 2**k, j)] = MLPClassifier(hidden_layer_sizes=(2**i, 2**k), activation=j)
    print(f"{i}/7 done")


1/7 done
2/7 done
3/7 done
4/7 done
5/7 done
6/7 done
7/7 done


In [ ]:
k = 10
shape = xtr.shape
print(shape[0]/k)
rows = shape[0]
xfolds = []
yfolds = []
lstr_acs_folds = [[] for _ in range(k)]
lste_acs_folds = [[] for _ in range(k)]
gstr_acs_folds = [[] for _ in range(k)]
gste_acs_folds = [[] for _ in range(k)]
nntr_acs_folds = [[] for _ in range(k)]
nnte_acs_folds = [[] for _ in range(k)]



for i in range(k):
    xfolds.append(xtr[i*(shape[0]//k): i*(shape[0]//k) + shape[0]//k])
    yfolds.append(ytr_noisy[i*(shape[0]//k): i*(shape[0]//k) + shape[0]//k])

for i in range(len(xfolds)):
    kx_test = xfolds[i]
    ky_test = yfolds[i]
    kx_train = np.concatenate([xfolds[j] for j in range(len(xfolds)) if j!= i])
    ky_train = np.concatenate([yfolds[j] for j in range(len(yfolds)) if j!= i])
    for j in lsvm:
        lsvm[j].fit(kx_train, ky_train)
        lstr_acs_folds[i].append((j, accuracy_score(ky_train, lsvm[j].predict(kx_train))))
        lste_acs_folds[i].append((j, accuracy_score(ky_test, lsvm[j].predict(kx_test))))
        print(f"{i}, {j}/{len(lsvm)} done")


    for j in gsvm:
        gsvm[j].fit(kx_train, ky_train)
        gstr_acs_folds[i].append((j, accuracy_score(ky_train, gsvm[j].predict(kx_train))))
        gste_acs_folds[i].append((j, accuracy_score(ky_test, gsvm[j].predict(kx_test))))
        print(f"{i}, {j}/{len(gsvm)} done")

    for j in neural:
        neural[j].fit(kx_train, ky_train)
        nntr_acs_folds[i].append((j, accuracy_score(ky_train, neural[j].predict(kx_train))))
        nnte_acs_folds[i].append((j, accuracy_score(ky_test, neural[j].predict(kx_test))))
        print(f"{i}, {j}/{len(neural)} done")

    print(f"{i + 1}/{k} done")



6000.0
0, 0.0001/10 done
